In [41]:
# %load_ext autoreload
# %autoreload 2

#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time    : 2022/11/08 15:54
# @Author  : Wang Yujia
# @File    : basicinfo_GT_models.ipynb
# @Description : 关于GT模型的计算结果的基本信息

# 0. What for
1. 关于GT模型的计算结果的基本信息

# 1. Preparations
## 1.1 全局设置

In [84]:

features_GT = ['product_id','bidincrement','bidfee','retail']

import numpy as np
import csv
import pandas as pd
from visdom import Visdom
from tqdm.notebook import tqdm


# viz = Visdom(env='P',use_incoming_socket=False)

## 1.2 读取data

In [148]:
# P in info_insym
P_1_path = "../data/info_asymm/results/asc_symmetry/GT_asc_symmetry_2_P.csv"
P_2_path = "../dataSA_PT/results/PT_all1288_P.csv"

P_1 = pd.read_csv(P_1_path, encoding="utf-8")
P_2 = pd.read_csv(P_2_path, encoding="utf-8")

FileNotFoundError: [Errno 2] No such file or directory: '../dataSA_PT/results/results/PT_all1288_P.csv'

# 2. Info about P which is the input of NNs
## 2.1 How many auction settings are included in 'info_insymm', 'GT'

In [107]:
print(f"In 'info_insymm', there is *{P_1.shape}*")
print(f"In 'GT', there is *{P_2.shape}*")

In 'info_insymm', there is *(612, 4)*
In 'GT', there is *(612, 5)*


# 3. Info about dataset
## 3.1 How many ‘settings’ in the dataset
1. dataset指的是samples数量在16之上的所有的ascending-price auctions,GT的两个model用这些data做了generate的过程
2. 根据settings含义的不同，需要有不同的统计

In [151]:
data_path = "../data/info_asymm/datawithnp_asc_symmetry_2_selected.csv"
data = pd.read_csv(data_path, encoding="utf-8")

setting_1 = ['retail','bidfee','bidincrement']
setting_2 = ['product_id','retail','bidfee','bidincrement']

data_1 = data.groupby(setting_1)
data_2 = data.groupby(setting_2)

print(f"按照setting={setting_1}, 一共有 *{len(data_1)}*")
print(f"按照setting={setting_2}, 一共有 *{len(data_2)}*")

按照setting=['retail', 'bidfee', 'bidincrement'], 一共有 *612*
按照setting=['product_id', 'retail', 'bidfee', 'bidincrement'], 一共有 *1303*


## 3.2 Are the dataset in the same size in the amount of setting?
1. 2个GT models用的dataset有微小不同，他们的settings数目是一样的吗: YES

In [152]:
data_pt_path = "../data/SA_PT/datawithnp_PT_selected.csv"
data_pt = pd.read_csv(data_pt_path, encoding="utf-8")

data_pt_1 = data_pt.groupby(setting_1)
data_pt_2 = data_pt.groupby(setting_2)

print(f"按照setting={setting_1}, 一共有 *{len(data_pt_1)}*")
print(f"按照setting={setting_2}, 一共有 *{len(data_pt_2)}*")

按照setting=['retail', 'bidfee', 'bidincrement'], 一共有 *612*
按照setting=['product_id', 'retail', 'bidfee', 'bidincrement'], 一共有 *1303*


# 4. Info about result ‘P’
## 4.1 the length of ‘P’ of different GT models the same? Yes
1. 比较2个model的P的长度是否一样（一样的话方便画图）：
2. 注意这里的P是**未经过K筛选**的，

In [29]:
# GT model-0
filename_P_0 = "../data/SA_PT/results/PT_all1303_P.csv"
# GT model-1
filename_P_1 = "../data/info_asymm/results/asc_symmetry/GT_asc_symmetry_2_woKP.csv"

# GT model-2
filename_P_2 = "../data/SA_PT/results/PT_all1303_oneforall_P.csv"

# read data
P_0 = pd.read_csv(filename_P_0, encoding="utf-8")
P_1 = pd.read_csv(filename_P_1, encoding="utf-8")
P_2 = pd.read_csv(filename_P_2, encoding="utf-8")


In [5]:
# transform str 'P' into narray 'P'
def transform(str):
    a = np.array(np.mat(str))
    d = a.flatten()
    return d

In [155]:
print(f"GT model-1 包括了 *{P_1.shape[0]}* 组setting")
print(f"GT model-2 包括了 *{P_2.shape[0]}* 组setting")

for i in range(0,P_1.shape[0]):
    # transform str into narray
    p_1_i = transform(P_1.loc[i,'P'])
    p_2_i = transform(P_2.loc[i,'P'])
    assert(len(p_1_i) == len(p_2_i),f"{i}: P不等长!")
print("两个GTmodel的'P'等长")

<>:8: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:8: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:8: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<ipython-input-155-8009a2cabfc1>:8: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(p_1_i) == len(p_2_i),f"{i}: P不等长!")


GT model-1 包括了 *612* 组setting
GT model-2 包括了 *612* 组setting
两个GTmodel的'P'等长


## 4.2 compare ‘P’ generated by different GT models [2 lines]
1. 比较两个P的图像，画在visdom中

In [ ]:
# 添加,use_incoming_socket=False之后似乎没办法append？
env_str = "compare_P"
viz = Visdom(env = "compare_P")

In [ ]:
# P_1.shape[0]
for i in range(0,P_1.shape[0]):
    # transform str into narray
    p_1_i = transform(P_1.loc[i,'P'])
    p_2_i = transform(P_2.loc[i,'P'])

    v = P_1.loc[i,'retail']
    b = P_1.loc[i,'bidfee']
    d = P_1.loc[i,'bidincrement']

    viz.line(Y = p_1_i, X = np.arange(0,len(p_1_i)), win = "P_"+str(i), env = env_str,update = 'append',name="model-1",
             opts= dict(title = f'P_{i}_v={v}_b={b}_d={d}',showlegend=True))
    viz.line(Y = p_2_i, X = np.arange(0,len(p_2_i)), win = "P_"+str(i), env = env_str,update = 'append',name="model-2")

print("Done")

In [ ]:
# 保存整个环境
viz.save(envs=['compare_P'])

## 4.3 compare 'P' under different params [3 lines]
1. 这次画3条曲线，2条和上面一样，多一条表示的是对每个settings做一次infer得到params，这个params下generate出来的P曲线
2. 因此有1303而不是612组“settings”

In [46]:
P_1.head()

,bidincrement,bidfee,retail,P
0,0.15,0.75,169.99,"[0.004412024236719825, 0.004396437716806756, 0..."
1,0.15,0.75,499.99,"[0.001500030000600061, 0.001498229388403341, 0..."
2,0.15,0.75,299.99,"[0.0025000833361111807, 0.0024950805012603767,..."
3,0.15,0.75,89.99,"[0.008334259362151375, 0.00827859868074787, 0...."
4,0.15,0.75,59.99,"[0.012502083680613452, 0.012376728563495022, 0..."


从1303那个dataset中，根据key_i找到对应的P

In [78]:
def get_P_from_i(i):
    v = P_0.loc[i,'retail']
    b = P_0.loc[i,'bidfee']
    d = P_0.loc[i,'bidincrement']

    p_1 = P_1[(P_1.loc[:,'retail'] == v) & (P_1.loc[:,'bidfee'] == b) & (P_1.loc[:,'bidincrement'] == d)]
    p_2 = P_2[(P_2.loc[:,'retail'] == v) & (P_2.loc[:,'bidfee'] == b) & (P_2.loc[:,'bidincrement'] == d)]

    p_1_arr = transform(p_1.P.item())
    p_2_arr = transform(p_2.P.item())

    return (p_1_arr,p_2_arr)

In [80]:
env_str = "compare_P_2"
viz = Visdom(env=env_str)

Setting up a new session...


In [86]:
for i in tqdm(range(0,P_0.shape[0])):
    p_0_i = transform(P_0.loc[i,'P'])
    # get transformed P from key_i
    p_1_i,p_2_i = get_P_from_i(i)

    v = P_0.loc[i,'retail']
    b = P_0.loc[i,'bidfee']
    d = P_0.loc[i,'bidincrement']

    # assert(len(p_0_i) == len(p_1_i),"Not in the same length!")

    viz.line(Y = p_0_i, X = np.arange(0,len(p_0_i)), win = "P_"+str(i), env = env_str,update = 'append',name="model-0",
             opts= dict(title = f'P_{i}_v={v}_b={b}_d={d}',showlegend=True))
    viz.line(Y = p_1_i, X = np.arange(0,len(p_1_i)), win = "P_"+str(i), env = env_str,update = 'append',name="model-1")
    viz.line(Y = p_2_i, X = np.arange(0,len(p_2_i)), win = "P_"+str(i), env = env_str,update = 'append',name="model-2")

print("Done")

  0%|          | 0/1303 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# 保存整个环境
viz.save(envs=env_str)

## 4.3 'P' in the target data
1. 如果setting=[v,b,d]的话，画‘P’没什么意义，因为对于实际数据，不止要考虑这3个设置，还要考虑product_id等